```{shinylive}
#| standalone: true
#| viewerHeight: 600
library(shiny)
library(bslib)
library(tidyverse)
library(ggplot2)
library(plotly)
library(sf)

# function for loading data
load_data <- function() {
  url <- "https://raw.githubusercontent.com/rasools/GDP_PPP/main/data/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_204/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_204.csv"
  read.csv(url, sep = ";", header = TRUE, skip = 4) %>%
    rename_with(~ gsub("X", "", .x)) %>%
    select(-c(3:44, 68)) %>%
    drop_na() %>%
    pivot_longer(cols = -c(1:2), names_to = "Year", values_to = "Value") %>%
    mutate(Year = as.numeric(Year)) %>%
    mutate(Value = ceiling(Value))
}

data <- load_data()

# Define UI for app that draws a histogram ----
ui <- page_sidebar(
  # App title ----
  title = "GDP PPP!",
  # Sidebar panel for inputs ----
  sidebar = sidebar(
    sliderInput("year", "Years", min = min(data$Year), max = max(data$Year), value = c(min(data$Year), max(data$Year)), step=1),
    selectInput("country", "Select Countries", choices = unique(data$Country.Name), selected = "Sweden", multiple = TRUE)
  ),
  
  plotOutput("line_plot"),
  plotOutput("world_heatmap")

)

server <- function(input, output) {
  
  # Load data
  data <- reactive(load_data())
  
  # Filter the data
  filtered_data_lineplot <- reactive({
    req(input$country, input$year)
    data() %>%
      filter(`Country.Name` %in% input$country,
             Year >= input$year[1],
             Year <= input$year[2])
  })
  
  # Create the line plot
  output$line_plot <- renderPlot({
    ggplot(filtered_data_lineplot(), aes(x = Year, y = Value, color = `Country.Name`)) +
      geom_line(na.rm = TRUE) +
      labs(title = "GDP PPP Over Time",
           x = "Year",
           y = "GDP PPP") +
      theme_minimal()
  })
  
  # Read the jeojson file for the world map
  world <- reactive({
    read_sf("https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json#")
  })
  
  # Filter the data
  filtered_data_world <- reactive({
    req(input$year)
    data() %>%
      filter(Year == input$year[2])
  })
    
  # Merge the world data with the filtered data
  hm_df <- reactive({
    left_join(world(), filtered_data_world(), by = c("id" = "Country.Code"))
  })
  
  # Create the world heatmap
  output$world_heatmap <- renderPlot({
    ggplot(hm_df(), aes(fill = Value)) +
      geom_sf() +
      scale_fill_viridis_c() +
      labs(title = paste("GDP PPP in", input$year[2]),
           x = "Longitude",
           y = "Latitude") +
      theme_minimal()
  })
  
}
# Run the application
shinyApp(ui = ui, server = server)
```